In [198]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from math import floor, ceil, inf
from random import randint, random, choice
import pprint 
from tqdm.auto import tqdm
import sys
import time 


## The *Nim* and *Nimply* classes

In [199]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [200]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    def full_rows(self) -> int:
        return sum(1 for r in self._rows if r != 0)
    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [201]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [202]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [203]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [204]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = choice(spicy_moves)
    return ply


#### Evolutionary algorithm FIRST ALGORITHM

In [205]:
def mutation(indiv: list, state :Nim) -> list: 
    index = randint(0, len(indiv)-1)
    ind = indiv[index].copy()
    max_int = state.rows[ind[0]]
    new_value = randint(1, max_int)
    ind[1] = new_value
    tmp = deepcopy(state)
    tmp.nimming(Nimply(ind[0], ind[1]))
    ind[2]=nim_sum(tmp)
    return ind

def mutation2(indiv: list, state :Nim) -> list:
    index = randint(0, len(indiv)-1)
    ind = indiv[index].copy()
    max_int = state.rows[ind[0]]
    new_value = max_int - 1 if max_int-1 != 0 else 1 #we try to delete the maximum number of elements - 1 (to leave at least one element in the line)
    ind[1] = new_value
    tmp = deepcopy(state)
    tmp.nimming(Nimply(ind[0], ind[1]))
    ind[2]=nim_sum(tmp)
    return ind

def mutation3(indiv: list, state :Nim) -> list:
    current_nim_sum = nim_sum(state)
    index = randint(0, len(indiv)-1) if len(indiv) > 1 else 0
    
    ind = indiv[index].copy()
    elements_in_line = state.rows[ind[0]]
    rule = current_nim_sum ^ elements_in_line
    
    if rule < elements_in_line:
        #print(f"sub in the line {ind[0]}, the value {elements_in_line - rule} before it has {elements_in_line}")
        new_value = elements_in_line - rule   
    else :
        new_value = randint(1, elements_in_line) if elements_in_line > 1 else 1
    tmp = deepcopy(state)
    tmp.nimming(Nimply(ind[0], new_value))
    ind[2] = nim_sum(tmp)
    ind[1] = new_value
    return ind



def crossover(ind_one : list, ind_two : list, state : Nim) -> list :
    #The idea is to take the row of the first individual and the value of the second individual 
    # the opposite operation if the first one is not possible
    # and if both are not possible, we keep the first individual
    #calcualte the max value of the row of the first individual
    max_ind_one = state.rows[ind_one[0]]

    if ind_two[1] <= max_ind_one:
        #print("change 1")
        #print(f"setting in the line {ind_one[0]}, the value {ind_two[1]}")
        new_ind = [ind_one[0], ind_two[1]]
        tmp = deepcopy(state)
        tmp.nimming(new_ind)
        new_ind.append(nim_sum(tmp))
        return new_ind
    else :
        max_ind_two = state.rows[ind_two[0]]
        if ind_one[1] <= max_ind_two:
            #print("change 2")
            #print(f"setting in the line {ind_two[0]}, the value {ind_one[1]}")
            new_ind = [ind_two[0], ind_one[1]]
            tmp = deepcopy(state)
            tmp.nimming(new_ind)
            new_ind.append(nim_sum(tmp))
            return new_ind
        else :
            return ind_one.copy()

def tournament_selection(population : list, k :int = 2, sorted_population : int =2 ) -> list :
    selected = list()
    for _ in range(k):
        selected.append(population[randint(0, len(population)-1)])
    selected = np.array(selected)
    sorted_index = np.argsort(selected[:,sorted_population])
    return (selected[sorted_index])[0]


def evolutionary_strategies(state : Nim, generation : int = 3, percentage_population : float = 0.8, mutation_rate : float = 0.10, sorted_population : int = 2) : 
#sorted population = 1 -> sort by min number of objects taken
#sorted population = 2 -> sort by nim sum
    analysis = analize(state)['possible_moves'] ##retrieves all possible moves and their nim sum
    population_size=ceil(len(analysis.keys())  * percentage_population) ##select random possible moves

    offspring_size = ceil(population_size/2)
    ##select random possible moves
    #p = list(set([ randint(0, len(analysis.keys())-1) for _ in range(population_size) ]))
    p = list()
    count = 0
    while count < population_size:
        tmp = randint(0, len(analysis.keys())-1)
        if tmp not in p:
            p.append(tmp)
            count += 1

    population  = []

    for key, value in analysis.items():
        population.append(list(key) + [value if value != 0 else sys.maxsize])
    population = np.array(population)[p]
    sorted_index = np.argsort(population[:,sorted_population])
    population = list(population[sorted_index])
    offspring = list()
    for _ in range(generation) :
        for index in range(offspring_size):
            if random() < mutation_rate : 
                new_ind = mutation(population, state=state)
            else :
                new_ind_one = tournament_selection(population , k=3, sorted_population= sorted_population)
                new_ind_two = tournament_selection(population, k=3, sorted_population= sorted_population)
                new_ind = crossover(new_ind_one, new_ind_two ,state=state) 
            offspring.append(new_ind)
        population = np.vstack([population, offspring])
        sorted_index = np.argsort(population[:,sorted_population])
        population = list(population[sorted_index])[:population_size]
    #print("deleting",population[0][0], population[0][1])
    return Nimply(int(population[0][0]), int(population[0][1]))

    

### EA second algorithm 
using a comma strategy, that means the offspring completely replaces the parent population

In [206]:

def evolutionary_strategies_comma(state : Nim, generation : int = 3, percentage_population : float = 0.8, mutation_rate : float = 0.10, sorted_population : int = 2) : 
#sorted population = 1 -> sort by min number of objects taken
#sorted population = 2 -> sort by nim sum
    analysis = analize(state)['possible_moves'] ##retrieves all possible moves and their nim sum
    population_size=ceil(len(analysis.keys())  * percentage_population) ##select random possible moves

    offspring_size = ceil(population_size)
    ##select random possible moves
    #p = list(set([ randint(0, len(analysis.keys())-1) for _ in range(population_size) ]))
    p = list()
    count = 0
    while count < population_size:
        tmp = randint(0, len(analysis.keys())-1)
        if tmp not in p:
            p.append(tmp)
            count += 1

    population  = []

    for key, value in analysis.items():
        population.append(list(key) + [value if value != 0 else sys.maxsize])
    population = np.array(population)[p]
    sorted_index = np.argsort(population[:,sorted_population])
    population = list(population[sorted_index])
    offspring = list()
    for _ in range(generation) :
        for index in range(offspring_size):
            if random() < mutation_rate : 
                new_ind = mutation(population, state=state)
            else :
                new_ind_one = tournament_selection(population , k=15, sorted_population= sorted_population)
                new_ind_two = tournament_selection(population, k=15, sorted_population= sorted_population)
                new_ind = crossover(new_ind_one, new_ind_two ,state=state) 
            offspring.append(new_ind)
        population = np.array(offspring)
        sorted_index = np.argsort(population[:,sorted_population])
        population = list(population[sorted_index])
    #print("deleting",population[0][0], population[0][1])
    return Nimply(int(population[0][0]), int(population[0][1]))

### Matches 

Select the number of matches to play between 2 players.

In [207]:
NUM_MATCHES = 100
wins = [0, 0]
sequence = []
K = 7
logging.getLogger().setLevel(logging.INFO)

strategy = ( evolutionary_strategies,optimal)
#logging.info(f"init : {nim}")
player = 0
generation = [3, 5, 10, 15, 20,40,80,100]

for gen in range(len(generation)):
    with tqdm(total=NUM_MATCHES) as pbar:
        for i in range(NUM_MATCHES):
            nim = Nim(K) 
            while nim:
                if strategy == evolutionary_strategies:
                    ply = strategy[player](nim, generation=generation[gen], percentage_population=0.8, mutation_rate=0.10, sorted_population=2)                   
                else: 
                    ply = strategy[player](nim)
                #logging.info(f"ply: player {player} plays {ply}")
                #print(f"ply: player {player} plays {ply}")
                nim.nimming(ply)
                #logging.info(f"status: {nim}")
                player = 1 - player
            #print(f"winner: player {player}")
            wins[player] += 1
            sequence.append(player)
            pbar.update(1)
        
        print(f"wins : {wins} with generation {generation[gen]} ")
        print(f"Percentage : player 0 {wins[0]/NUM_MATCHES}, player 1 {wins[1]/NUM_MATCHES}")
        wins = [0, 0]
# for i in range(NUM_MATCHES):
#     print('player : ', sequence[i]) 


100%|██████████| 100/100 [00:03<00:00, 29.50it/s]


wins : [55, 45] with generation 3 
Percentage : player 0 0.55, player 1 0.45


100%|██████████| 100/100 [00:02<00:00, 34.11it/s]


wins : [60, 40] with generation 5 
Percentage : player 0 0.6, player 1 0.4


100%|██████████| 100/100 [00:02<00:00, 34.09it/s]


...

wins : [53, 47] with generation 40 
Percentage : player 0 0.53, player 1 0.47


100%|██████████| 100/100 [00:03<00:00, 31.16it/s]


wins : [55, 45] with generation 80 
Percentage : player 0 0.55, player 1 0.45


100%|██████████| 100/100 [00:03<00:00, 32.21it/s]

wins : [52, 48] with generation 100 
Percentage : player 0 0.52, player 1 0.48


This first part has been done previously i discovered that the nim-sum should not be used in the Evolutionary approach

# ES 

In [208]:
import inspect

class Rule : 
    def __init__(self , condition : callable , action : Nimply, weight : int =0.5) : 
        self.condition = condition
        self.action = action
        self.weight = weight

    def update_rule_weight(self, improve :bool = True) : 
        if improve :     
            self.weight *= 1.00001
        else :
            self.weight /= 1.00001

    def __str__(self) :
        return f"{inspect.getsource(self.condition)} -> {self.action} : {self.weight}"

class EsNimAgent : 
    def __init__(self, rules : list[Rule], played : int = 0, won : int = 0):
        self.rules = rules
        self.match_played = played
        self.match_won = won

    def win_match(self, win :bool = True) : 
        self.match_played += 1
        if win : 
            self.match_won += 1
        self.update_rules_weight(win)

    def update_rules_weight(self, win :bool = True) :
        '''
        A win means that the rules can be assumed as good rules, so we increase their weight
        A loss means that the rules are not good, so we decrease their weight 
        '''
        for rule in self.rules : 
            rule.update_rule_weight(win)
        

    def agent_fitness(self) : 
        if self.match_played == 0 : 
            return 0
        return self.match_won/self.match_played
    
    def move_selection(self, state : Nim) -> Nimply:
        possible_moves = list()
        for rule in self.rules :
            if rule.condition(state) and rule.weight > 0 \
            and rule.action.num_objects <= state.rows[rule.action.row] : ## to take the correct rule with the highest weight
                possible_moves.append([rule.action, rule.weight])
                #return rule.action
        if len(possible_moves) == 0 :
            if random() < 0.15 :
                self.update_rules_weight(False) ##the agent is penalized if it cannot find a rule to apply 
                #print("random penality") 
            return pure_random(state)
        else : 
            possible_moves.sort(key = lambda x : x[1], reverse = True) ##sort for the weight
            return possible_moves[0][0]


## Functions for the initialization of the ES

In [209]:
import pprint 

###initialization of my rules plus all possible moves associated

def all_possible_moves(state : Nim) -> list : 
    return [Nimply(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
   

def condition_for_rule(number_of_row : int) -> list[callable] :
    possible_rules = list()
    ### odd number of elements
    ##random element
    possible_rules.append(lambda state:  state.rows[randint(0, number_of_row-1)] > 1 and state.rows[randint(0, number_of_row-1)] > 1)

    ###if there are many element or not
    possible_rules.append(lambda state:   sum(state.rows) >  len(state.rows) * 2) ##many elements
    possible_rules.append(lambda state:   sum(state.rows) <= len(state.rows) * 2) ##few elements

    ##one possible final move
    possible_rules.append(lambda state:  sum(state.rows) <= 2) 
    possible_rules.append(lambda state:  sum(state.rows) ==3 and state.full_rows() == 2) 

    ###if there are many full rows or not
    possible_rules.append(lambda state:  state.full_rows() >= ceil(len(state.rows)/2)) ##many full rows
    possible_rules.append(lambda state:  state.full_rows() < ceil(len(state.rows)/2))
    ##add more rules 
    # possible_rules.append(lambda state: nim_sum(state) != 0)
    return possible_rules

def global_set_rules(conditions_for_rules : list, moves : list, fixed_weight : float = 0.5) -> list :
    global_set_of_rules = list()
    for condition in conditions_for_rules : 
        for move in moves : 
            global_set_of_rules.append(Rule(condition, move, fixed_weight))
    return global_set_of_rules  


### start the ES code

In [210]:
def create_population (global_set_of_rules : list, population_size : int = 10, numbers_of_rules : int = 5) -> list : 
    population = list()
    for _ in range(population_size) : 
        one_element = list()
        for __ in range(numbers_of_rules) : 
            choosen_rule = choice(global_set_of_rules)
            #print(choosen_rule)
            one_element.append(choosen_rule)
        population.append(EsNimAgent(one_element))  
    return population


In [211]:

def play_a_game (agent_one , agent_two, nim : Nim) -> int :
    '''
    Play a game between two agents, return 0 if the first agent wins, 1 otherwise. 
    Randomly choose who starts.
    '''
    turn = randint(0,1)
    strategies = [agent_one, agent_two]
    while nim : 
        ply = strategies[turn](nim)
        nim.nimming(ply)
        turn = 1 - turn
    return turn

def mutation(agent_orig : EsNimAgent, set_rules : list) -> EsNimAgent : 
    agent = deepcopy(agent_orig)
    rules = list()
    for rule in agent.rules :
        if random() < 0.5:
            r = choice(set_rules)
            rules.append(r)
        else : 
            r = rule
            rules.append(r)
    return EsNimAgent(rules, agent.match_played, agent.match_won)

def one_cut_crossover(agent_one : EsNimAgent, agent_two : EsNimAgent) -> EsNimAgent :
    '''
    This function keep the first half of rules from the first parent and the second half of rules from the second parent
    '''
    new_rules = list()
    for i in range(len(agent_one.rules)) :
        if i < ceil(len(agent_one.rules)/2) :
            new_rules.append(agent_one.rules[i])
        else : 
            new_rules.append(agent_two.rules[i])
    return EsNimAgent(new_rules, ceil((agent_one.match_played + agent_two.match_played)/2), ceil((agent_one.match_won + agent_two.match_won)/2) )
    ##the first value is always 3 * num matches 
    

def survival_selection(population : list[EsNimAgent], population_size : int = 10) -> list[EsNimAgent] : 
    ##sort the population by fitness
    sorted_population = sorted(population, key=lambda x : x.agent_fitness(), reverse=True)
    ##it is necessary to reset the number of matches played and won
    for agent in sorted_population[:population_size]: 
        agent.match_played = 0
        agent.match_won = 0
    return sorted_population[:population_size]

def tournament_selection(population : list[EsNimAgent], k :int = 2) -> list :
    selected = list()
    for _ in range(k):
        selected.append(choice(population))
    ### the one with the highest fitness wins
    return max(selected, key=lambda x : x.agent_fitness())

def generate_new_generation(population : list[EsNimAgent], tournament_size : int = 5,offspring_size : int = 0 , mutation_rate : float = 0.1, \
                            set_rules : list = list()) :
    new_offspring = list()
    for _ in range(offspring_size) : 
        parent_one = tournament_selection(population, k=tournament_size)
        
        if random() < mutation_rate : 
            new_offspring.append(mutation(parent_one, set_rules=set_rules))

        else : #cross over
            parent_two = tournament_selection(population, k=tournament_size)
            new_offspring.append(one_cut_crossover(parent_one, parent_two))
    return new_offspring

###GENERATION STRATEGIES
def ES(population :list [EsNimAgent] , num_generation : int , \
       num_matches : int, num_rows : int , tournament_size : int ,\
          mutation_rate : float ,set_rules : list) -> EsNimAgent :
    t1 = time.time()
    for _ in range(num_generation):
        for agent in population : 
            #each agent plays against the std agent in the population
            for _ in range(num_matches) : 
                current_win = 0
                nim = Nim(num_rows)
                w = play_a_game(agent.move_selection, optimal, nim) #against the optimal strategy
                if w == 0 : 
                    current_win += 1
                nim = Nim(num_rows)
                w = play_a_game(agent.move_selection, pure_random, nim) #against the random strategy
                if w == 0 : 
                    current_win += 1
                nim = Nim(num_rows)
                w = play_a_game(agent.move_selection, gabriele, nim) #against the gabriele strategy
                if w == 0 : 
                    current_win += 1
                ####update the weight of the agent
                for _ in range(current_win) : 
                    agent.win_match(True)
                for _ in range(3-current_win) :
                    agent.win_match(False)
        POPULATION_SIZE = len(population)        
        offspring = generate_new_generation(population,tournament_size,POPULATION_SIZE, mutation_rate,set_rules)
        population.extend(offspring)
        population = survival_selection(population, POPULATION_SIZE)
        #print(f"generation {_} completed")
    t2 = time.time()
    print(f"ES time {t2-t1}")
    return population[0]



In [240]:
##first setting

NUM_ROWS = 4
POPULATION_SIZE = 10
NUM_RULES = 10
GENERATIONS = 10
TOURNAMENT_SIZE = 4
NUM_MATCHES = 100
MUTATION_RATE = 0.1

nim = Nim(NUM_ROWS)
all_moves = all_possible_moves(nim)
conditions = condition_for_rule(NUM_ROWS)
set_rules = global_set_rules(conditions, all_moves)
population = create_population(set_rules, population_size=POPULATION_SIZE, numbers_of_rules=NUM_RULES)

last_remain_one = ES(population,GENERATIONS, NUM_MATCHES,NUM_ROWS,TOURNAMENT_SIZE, MUTATION_RATE, set_rules)


ES time 13.7972412109375


In [ ]:
NUM_ROWS = 7
POPULATION_SIZE = 20
NUM_RULES = 20
GENERATIONS = 50
TOURNAMENT_SIZE = 8
NUM_MATCHES = 50
MUTATION_RATE = 0.1

nim = Nim(NUM_ROWS)
all_moves = all_possible_moves(nim)
conditions = condition_for_rule(NUM_ROWS)
set_rules = global_set_rules(conditions, all_moves)
population = create_population(set_rules, population_size=POPULATION_SIZE, numbers_of_rules=NUM_RULES)

last_remain = ES(population,GENERATIONS, NUM_MATCHES,NUM_ROWS,TOURNAMENT_SIZE, MUTATION_RATE,set_rules)

In [ ]:
NUM_ROWS = 5
POPULATION_SIZE = 20
NUM_RULES = 15
GENERATIONS = 80
TOURNAMENT_SIZE = 10
NUM_MATCHES = 100
MUTATION_RATE = 0.1
TEST_MATCHES = 100

nim = Nim(NUM_ROWS)
all_moves = all_possible_moves(nim)
conditions = condition_for_rule(NUM_ROWS)
set_rules = global_set_rules(conditions, all_moves)
population = create_population(set_rules, population_size=POPULATION_SIZE, numbers_of_rules=NUM_RULES)

last_remain_two = ES(population,GENERATIONS, NUM_MATCHES,NUM_ROWS,TOURNAMENT_SIZE, MUTATION_RATE,set_rules)


This section is to test the diffent training setup 

In [239]:
best_agent = last_remain ###here we set the best agent trained with specific parameters
wins = [0, 0]
sequence = []
strategy = [ best_agent.move_selection,optimal]
TEST_MATCHES = 1000
for i in range(TEST_MATCHES):
    turn = randint(0,1)
    nim = Nim(NUM_ROWS) 
    while nim:
        ply = strategy[turn](nim)
        nim.nimming(ply)
        turn = 1 - turn
    wins[turn] += 1
    sequence.append(turn)      
    
print(f"wins : {wins} ")
print(f"Percentage : player 0 {wins[0]/TEST_MATCHES}, player 1 {wins[1]/TEST_MATCHES}")
print(f"sequence : {sequence}")

wins : [263, 737] 
Percentage : player 0 0.263, player 1 0.737
sequence : [0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1